### Load project

In [1]:
from pathlib import Path

from mynd.backend import metashape as metashape
from mynd.collections import GroupID
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


CACHE: Path = Path("/data/kingston_snv_01/acfr_point_clouds")
INPUT_DIR: Path = Path("/data/kingston_snv_01/acfr_metashape_projects_dev")
OUTPUT_DIR: Path = Path(
    "/data/kingston_snv_01/acfr_metashape_projects_registered"
)


# qdc5ghs3 - qdc5ghs3_20100430_024508
# qdch0ftq - qdch0ftq_20100428_020202
# qdchdmy1 - qdchdmy1_20110416_005411
# r7jjskxq - r7jjskxq_20101023_210332
# r23m7ms0 - r23m7ms0_20100606_001908
# r29mrd5h - r29mrd5h_20090612_225306
# r234xgje - r234xgje_20100604_230524
# r23685bc - r23685bc_20100605_021022


SITE_ID: str = "qdch0ftq"
target: GroupID = GroupID(key=0, label="qdch0ftq_20100428_020202")


INPUT_PROJECT: Path = INPUT_DIR / f"{SITE_ID}_dense_with_metadata.psz"
DESTINATION_PROJECT: Path = (
    OUTPUT_DIR / f"{SITE_ID}_registered_with_metadata.psz"
)


assert (
    INPUT_PROJECT != DESTINATION_PROJECT
), "source and destination project cannot be the same!"

assert CACHE.exists(), f"directory does not exist: {CACHE}"
assert OUTPUT_DIR.exists(), f"directory does not exist: {OUTPUT_DIR}"
assert INPUT_PROJECT.exists(), f"project does not exist: {INPUT_PROJECT}"


match metashape.load_project(INPUT_PROJECT):
    case Ok(path):
        logger.info(path)
    case Err(error_message):
        logger.info(error_message)
    case _:
        raise NotImplementedError

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
LoadProject: path = /data/kingston_snv_01/acfr_metashape_projects_dev/qdch0ftq_dense_with_metadata.psz


2024-11-12 05:04:52.727 | INFO | Line   45 (3516103728.py): loaded document /data/kingston_snv_01/acfr_metashape_projects_dev/qdch0ftq_dense_with_metadata.psz successfully


loaded project in 58.7571 sec


In [2]:
from mynd.collections import GroupID
from mynd.geometry import PointCloudLoader
from mynd.registration import (
    RegistrationPipeline,
    build_registration_pipeline,
)

from mynd.io import read_config
from mynd.utils.log import logger


# Retrieve dense point clouds
retrieval_result: Result = metashape.dense_services.retrieve_dense_point_clouds(
    cache=CACHE,
    overwrite=True,
)

if retrieval_result.is_err():
    logger.error(retrieval_result.err())

point_cloud_loaders: dict[GroupID, PointCloudLoader] = retrieval_result.ok()

ExportPointCloud: path = /data/kingston_snv_01/acfr_point_clouds/qdch0ftq_20100428_020202.ply, save_point_classification = off, save_point_confidence = off, save_comment = off
point cloud size: 9878114 points
ExportPointCloud: path = /data/kingston_snv_01/acfr_point_clouds/qdch0ftq_20110415_020103.ply, save_point_classification = off, save_point_confidence = off, save_comment = off
point cloud size: 12338433 points
ExportPointCloud: path = /data/kingston_snv_01/acfr_point_clouds/qdch0ftq_20120430_002423.ply, save_point_classification = off, save_point_confidence = off, save_comment = off
point cloud size: 11389618 points
ExportPointCloud: path = /data/kingston_snv_01/acfr_point_clouds/qdch0ftq_20130406_023610.ply, save_point_classification = off, save_point_confidence = off, save_comment = off
point cloud size: 12036098 points


In [8]:
from mynd.geometry import PointCloud
from mynd.registration import RegistrationResult
from mynd.registration import RegistrationBatch, register_batch
from mynd.registration import (
    RegistrationIndex,
    generate_indices_cascade,
    generate_indices_one_way,
)

from mynd.registration import log_registration_result
from mynd.visualization import visualize_registration


def batch_callback(
    target: GroupID, source: GroupID, result: RegistrationResult
) -> None:
    """Callback for registration."""
    logger.info("")
    logger.info(f"Target: {target.label}")
    logger.info(f"Source: {source.label}")
    log_registration_result(result=result)
    logger.info("")


CONFIG_FILE: Path = Path(
    "/home/martin/dev/mynd/config/register_advanced_highres.toml"
)

config: dict = read_config(CONFIG_FILE).unwrap()

pipeline: RegistrationPipeline = build_registration_pipeline(
    config.get("registration")
)


batch: RegistrationBatch = RegistrationBatch[GroupID](point_cloud_loaders)

for key in batch.keys():
    logger.info(f"Key: {key.label}")

index_strategy: str = "one-way"

match index_strategy:
    case "one-way":
        indices: list[RegistrationIndex] = generate_indices_one_way(
            target, batch.keys()
        )
    case "cascade":
        indices: list[RegistrationIndex] = generate_indices_cascade(
            batch.keys()
        )
    case _:
        raise NotImplementedError

logger.info("")
logger.info("Performing batch registration...")
registration_results: list[RegistrationBatch.PairResult] = register_batch(
    batch, pipeline, indices, callback=batch_callback
)
logger.info("Batch registration done!")
logger.info("")

2024-11-12 06:07:00.061 | INFO | Line   37 (2378660065.py): Key: qdch0ftq_20100428_020202
2024-11-12 06:07:00.062 | INFO | Line   37 (2378660065.py): Key: qdch0ftq_20110415_020103
2024-11-12 06:07:00.062 | INFO | Line   37 (2378660065.py): Key: qdch0ftq_20120430_002423
2024-11-12 06:07:00.063 | INFO | Line   37 (2378660065.py): Key: qdch0ftq_20130406_023610
2024-11-12 06:07:00.063 | INFO | Line   49 (2378660065.py): 
2024-11-12 06:07:00.063 | INFO | Line   50 (2378660065.py): Performing batch registration...
registering batch...:   0%|                                                                                                                                                           | 0/3 [00:00<?, ?it/s]2024-11-12 06:14:11.335 | INFO | Line   18 (2378660065.py): 
2024-11-12 06:14:11.336 | INFO | Line   19 (2378660065.py): Target: qdch0ftq_20100428_020202
2024-11-12 06:14:11.336 | INFO | Line   20 (2378660065.py): Source: qdch0ftq_20110415_020103
2024-11-12 06:14:11.337 | INFO | Li

### Get consensus estimate with pose graph optimization

In [4]:
# TODO: Build pose graph
# TODO: Optimize pose graph

### Visualize registration results

In [9]:
visualize: bool = True
if visualize:
    for registration in registration_results:
        target_loader: PointCloudLoader = batch.get(registration.target)
        source_loader: PointCloudLoader = batch.get(registration.source)

        target_cloud: PointCloud = target_loader().unwrap()
        source_cloud: PointCloud = source_loader().unwrap()

        visualize_registration(
            target=target_cloud,
            source=source_cloud,
            transformation=registration.result.transformation,
            title=f"{registration.source.label}",
        )

### Update chunk transforms

In [10]:
import Metashape as ms
import numpy as np

chunks: dict[GroupID, ms.Chunk] = {
    GroupID(chunk.key, chunk.label): chunk
    for chunk in metashape.context._backend_data.get("document").chunks
}


results: dict[GroupID, RegistrationResult] = results.get(target)


# TODO: Move to metashape backend
target_chunk: ms.Chunk = chunks.get(target)
for source, registration in results.items():
    source_chunk: ms.Chunk = chunks.get(source)

    logger.info(
        f"Aligning chunks - target: {target_chunk.label}, source: {source_chunk.label}"
    )
    metashape.align_chunks(target_chunk, source_chunk, registration)

2024-11-12 06:29:35.318 | INFO | Line   25 (3327945508.py): Aligning chunks - target: qdch0ftq_20100428_020202, source: qdch0ftq_20110415_020103
2024-11-12 06:29:35.319 | INFO | Line   25 (3327945508.py): Aligning chunks - target: qdch0ftq_20100428_020202, source: qdch0ftq_20120430_002423
2024-11-12 06:29:35.320 | INFO | Line   25 (3327945508.py): Aligning chunks - target: qdch0ftq_20100428_020202, source: qdch0ftq_20130406_023610


### TODO: Save project to file

In [11]:
assert (
    INPUT_PROJECT != DESTINATION_PROJECT
), "source and destination project cannot be the same!"

match metashape.save_project(DESTINATION_PROJECT):
    case Ok(path):
        logger.info(f"saved project to: {path}")
    case Err(message):
        logger.error(message)
    case _:
        raise NotImplementedError

SaveProject: path = /data/kingston_snv_01/acfr_metashape_projects_registered/qdch0ftq_registered_with_metadata.psz


2024-11-12 06:31:11.122 | INFO | Line    7 (3199932015.py): saved project to: /data/kingston_snv_01/acfr_metashape_projects_registered/qdch0ftq_registered_with_metadata.psz


saved project in 93.1391 sec
